In [91]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [92]:
df = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_5337/3668100114.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/SBAnational_clean.csv")


In [93]:
df = df[['NAICS', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob','FranchiseCode', 'UrbanRural', 'MIS_Status']]

In [94]:
# Séparer les données en ensembles d'entraînement et de test
X = df.drop(columns=["MIS_Status"])
y = df["MIS_Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.2)

# Encodage des variables catégorielles
encoder = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = encoder.fit_transform(X[col])

# Entraînement du modèle
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluation du modèle sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Importance des variables
importances = rf.feature_importances_

Accuracy: 80.50%


In [95]:
rf.score(X_train,y_train)

0.8887289930800735

In [96]:
accuracy

0.804971049286824

In [97]:
importances

array([0.54881246, 0.09677483, 0.010981  , 0.07623475, 0.09339248,
       0.12445546, 0.04934903])

In [98]:
X

,NAICS,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural
0,451120,4,2.0,0,0,1,0
1,722410,2,2.0,0,0,1,0
2,621210,7,1.0,0,0,1,0
3,0,2,1.0,0,0,1,0
4,0,14,1.0,7,7,1,0
...,...,...,...,...,...,...,...
885120,451120,6,1.0,0,0,1,0
885121,451130,6,1.0,0,0,1,0
885122,332321,26,1.0,0,0,1,0
885123,0,6,1.0,0,0,1,0


Il semblerait que le NAICS n'a pas besoin d'être transformer dans un randomclassifier , no emp est pas mal , NewExist , a que tres peu d'importance , Retained job est "prenable" , FranchiseCode ne semble pas être important , peut être une transformation avant l'aurait améliorer  , urbane rural ne rapporte pas beaucoup non plus 

ON DOIT SE POSER LA QUESTION DE SAVOIR SI NOTRE ALGORITHME FAIT MIEUX QUE SI IL DIT NON/OUI A CHAQUE FOIS 

DE CE PETIT CALCUL ON EN DEDUIT QUE environ 17,5 des entreprise ne rembourse pas , par consequent si notre algorithme doit faire mieux que dire oui a chaque fois , il doit obtenir un score de +82,5% 

In [99]:
from sklearn.metrics import confusion_matrix

# Prédire les valeurs pour l'ensemble de test
y_pred = rf.predict(X_test)

# Obtenir la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)


[[  6808  24622]
 [  9903 135692]]


REV Line credit a rajouter

Faire un deuxième test pour savoir 


In [100]:
df1 = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_5337/1589796403.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("archive/SBAnational_clean.csv")


In [101]:
df1.columns

Index(['Unnamed: 0', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'DisbursementDate', 'DisbursementGross', 'BalanceGross',
       'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')

In [102]:
df1 = df1[[  'NAICS',   'RetainedJob', 'MIS_Status',"NoEmp","Term","UrbanRural"]]

df1 = df1[[  'NAICS',   'RetainedJob', 'MIS_Status',"NoEmp","Term"]]

In [103]:
df1['MIS_Status'] = df1['MIS_Status'].apply(lambda x: 1 if x == 'CHGOFF' else 0)

In [104]:
df1 = df1.dropna()

In [105]:
df1.columns

Index(['NAICS', 'RetainedJob', 'MIS_Status', 'NoEmp', 'Term', 'UrbanRural'], dtype='object')

In [106]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Séparer les données en ensembles d'entraînement et de test
X = df1.drop(columns=["MIS_Status"])
y = df1["MIS_Status"]

# Encodage des variables catégorielles
encoder = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = encoder.fit_transform(X[col])

# Entraînement du modèle
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluation du modèle sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Importance des variables
importances = rf.feature_importances_ 


Accuracy: 80.53%


In [107]:
y

0         0
1         0
2         0
3         0
4         0
         ..
885120    0
885121    0
885122    0
885123    1
885124    0
Name: MIS_Status, Length: 885125, dtype: int64

In [108]:
X

,NAICS,RetainedJob,NoEmp,Term,UrbanRural
0,451120,0,4,84,0
1,722410,0,2,60,0
2,621210,0,7,180,0
3,0,0,2,60,0
4,0,7,14,240,0
...,...,...,...,...,...
885120,451120,0,6,60,0
885121,451130,0,6,60,0
885122,332321,0,26,108,0
885123,0,0,6,60,0


In [109]:
importances

array([0.5577335 , 0.09438341, 0.01061643, 0.07325032, 0.08940387,
       0.12553941, 0.04907305])

In [110]:
rf.score(X_train,y_train)

0.8887261686202513

In [111]:
X

,NAICS,RetainedJob,NoEmp,Term,UrbanRural
0,451120,0,4,84,0
1,722410,0,2,60,0
2,621210,0,7,180,0
3,0,0,2,60,0
4,0,7,14,240,0
...,...,...,...,...,...
885120,451120,0,6,60,0
885121,451130,0,6,60,0
885122,332321,0,26,108,0
885123,0,0,6,60,0


In [112]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define the preprocessor 
preprocessor = make_column_transformer(
    (StandardScaler(), X.select_dtypes(include='number').columns),
    (OneHotEncoder(), X.select_dtypes(include='object').columns),
)

# Fit the preprocessor on the train data
X_train = preprocessor.fit_transform(X_train)

# Apply the preprocessor on the test data
X_test = preprocessor.transform(X_test)

# Training the model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluation du modèle sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Importance des variables
importances = rf.feature_importances_ 


ValueError: A given column is not a column of the dataframe

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare the data 
X = df1.drop(columns=["MIS_Status"])
y = df1["MIS_Status"]

# Define the preprocessor 
preprocessor = make_column_transformer(
    (OneHotEncoder(), X.select_dtypes(include='object').columns),
)

# Fit the preprocessor on the train data
X_train = preprocessor.fit_transform(X_train)

# Apply the preprocessor on the test data
X_test = preprocessor.transform(X_test)

# Training the model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluation of the model on the test set
accuracy = rf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Importance of variables
importances = rf.feature_importances_ 


ValueError: Found array with 0 feature(s) (shape=(708100, 0)) while a minimum of 1 is required by RandomForestClassifier.

In [ ]:
from sklearn.metrics import f1_score

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score: {:.2f}".format(f1))


F1 score: 0.75


In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Generate the classification report
classification_report(y_test, y_pred)


              precision    recall  f1-score   support

           0       0.94      0.96      0.95    148321
           1       0.78      0.72      0.75     31512

    accuracy                           0.92    179833
   macro avg       0.86      0.84      0.85    179833
weighted avg       0.91      0.92      0.92    179833

